In [5]:
# !pip install lightly

In [1]:
import torch
import torchvision
import lightly.models as models
import lightly.loss as loss
import lightly.data as data
import pytorch_lightning as pl
import math
import os
import shutil
os.environ["CUDA_VISIBLE_DEVICES"]="1,0"

## Seperate train and test set

In [4]:
# for logo2k and logo targetlist
# train_list = [x.strip('\n') for x in open('/home/l/liny/ruofan/PhishIntention/src/siamese_retrain/train_targets.txt').readlines()]
# test_list = [x.strip('\n') for x in open('/home/l/liny/ruofan/PhishIntention/src/siamese_retrain/test_targets.txt').readlines()]

# os.makedirs('../datasets/targetlist/train', exist_ok=True)
# os.makedirs('../datasets/targetlist/test', exist_ok=True)

# for data in train_list:
#     if not os.path.exists(os.path.join('../datasets/targetlist/train/', data.split('/')[0])):
#         os.makedirs('../datasets/targetlist/train/' + data.split('/')[0], exist_ok=True)
#     try:
#         shutil.copyfile(os.path.join('/home/l/liny/ruofan/PhishIntention/src/phishpedia/expand_targetlist/', data),
#                         os.path.join('../datasets/targetlist/train/', data))
#     except FileExistsError:
#         continue

# for data in test_list:
#     if not os.path.exists(os.path.join('../datasets/targetlist/test/', data.split('/')[0])):
#         os.makedirs('../datasets/targetlist/test/' + data.split('/')[0], exist_ok=True)
#     try:
#         shutil.copyfile(os.path.join('/home/l/liny/ruofan/PhishIntention/src/phishpedia/expand_targetlist/', data),
#                         os.path.join('../datasets/targetlist/test/', data))
#     except FileExistsError:
#         continue

In [21]:
# # for celebA
# train_list = [x.split(' ')[0] for x in open('../datasets/celebA/list_eval_partition.txt').readlines() \
#                       if x.strip().split(' ')[1] == '0']

# valid_list = [x.split(' ')[0] for x in open('../datasets/celebA/list_eval_partition.txt').readlines() \
#                       if x.strip().split(' ')[1] == '1']

# test_list = [x.split(' ')[0] for x in open('../datasets/celebA/list_eval_partition.txt').readlines() \
#                       if x.strip().split(' ')[1] == '2']


# os.makedirs('../datasets/celebA/train', exist_ok=True)
# os.makedirs('../datasets/celebA/valid', exist_ok=True)
# os.makedirs('../datasets/celebA/test', exist_ok=True)

# for data in train_list:
#     try:
#         shutil.copyfile(os.path.join('../datasets/celebA/img_align_celeba/img_align_celeba/', data),
#                         os.path.join('../datasets/celebA/train/', data))
#     except FileExistsError:
#         continue

# for data in valid_list:
#     try:
#         shutil.copyfile(os.path.join('../datasets/celebA/img_align_celeba/img_align_celeba/', data),
#                         os.path.join('../datasets/celebA/valid/', data))
#     except FileExistsError:
#         continue

# for data in test_list:
#     try:
#         shutil.copyfile(os.path.join('../datasets/celebA/img_align_celeba/img_align_celeba/', data),
#                         os.path.join('../datasets/celebA/test/', data))
#     except FileExistsError:
#         continue

In [3]:
# # CIFAR100
# def unpickle(file):
#     import pickle
#     with open(file, 'rb') as fo:
#         dict = pickle.load(fo, encoding='bytes')
#     return dict

In [11]:
# traindict = unpickle('../datasets/cifar-100-python/train')

# os.makedirs('../datasets/CIFAR100/train', exist_ok=True)
# os.makedirs('../datasets/CIFAR100/test', exist_ok=True)

# for j, data in enumerate(traindict[b'data']):
#     filename = traindict[b'filenames'][j].decode('utf-8')
#     from PIL import Image
#     im = Image.fromarray(np.transpose(np.reshape(data, (3,32,32)), (1,2,0)))
#     im.save(os.path.join('../datasets/CIFAR100/train', filename))

In [10]:
# len(set(traindict[b'fine_labels']))

In [62]:
# testdict = unpickle('../datasets/cifar-100-python/test')
# for j, data in enumerate(testdict[b'data']):
#     filename = testdict[b'filenames'][j].decode('utf-8')
#     from PIL import Image
#     im = Image.fromarray(np.transpose(np.reshape(data, (3,32,32)), (1,2,0)))
#     im.save(os.path.join('../datasets/CIFAR100/test', filename))

In [64]:
# len(os.listdir('../datasets/CIFAR100/test/'))

In [24]:
# # clothing
# import pandas as pd
# df = pd.read_csv('../datasets/clothing-dataset/images.csv')
# train_imgs = list(df['image'])[:5000]
# test_imgs = list(df['image'])[5000:]

In [36]:
# os.makedirs('../datasets/clothing-dataset/train', exist_ok=True)
# os.makedirs('../datasets/clothing-dataset/test', exist_ok=True)

# for data in train_imgs:
#     try:
#         shutil.copyfile(os.path.join('../datasets/clothing-dataset/images', data+'.jpg'),
#                         os.path.join('../datasets/clothing-dataset/train', data+'.jpg'))
#     except FileExistsError:
#         continue
#     except FileNotFoundError:
#         continue

# for data in test_imgs:
#     try:
#         shutil.copyfile(os.path.join('../datasets/clothing-dataset/images', data+'.jpg'),
#                         os.path.join('../datasets/clothing-dataset/test', data+'.jpg'))
#     except FileExistsError:
#         continue
#     except FileNotFoundError:
#         continue

## Create dataloader, model

In [67]:
# the collate function applies random transforms to the input images
collate_fn = data.ImageCollateFunction(input_size=32, cj_prob=0.5)

# create a dataset from your image folder
dataset = data.LightlyDataset(input_dir='../datasets/CIFAR100/train/')

# build a PyTorch dataloader
dataloader = torch.utils.data.DataLoader(
    dataset,                # pass the dataset to the dataloader
    batch_size=128,         # a large batch size helps with the learning
    shuffle=True,           # shuffling is important!
    collate_fn=collate_fn,
    drop_last=False)  # apply transformations to the input images

In [69]:
# use a resnet50 backbone
resnet = torchvision.models.resnet.resnet50()
resnet = torch.nn.Sequential(*list(resnet.children())[:-1])

# build the simsiam model
model = models.SimSiam(resnet, num_ftrs=2048)

# use a criterion for self-supervised learning
criterion = loss.SymNegCosineSimilarityLoss()

# get a PyTorch optimizer
optimizer = torch.optim.SGD(model.parameters(), lr=1e-0, weight_decay=1e-5)

## Training

In [ ]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'
model = torch.nn.DataParallel(model)
model.to(device)

avg_loss = 0.
avg_output_std = 0.
epochs = 300
out_dim = 2048

for e in range(epochs):

    for (x0, x1), _, _ in dataloader:

        # move images to the gpu
        x0 = x0.to(device)
        x1 = x1.to(device)

        # run the model on both transforms of the images
        # the output of the simsiam model is a y containing the predictions
        # and projections for each input x
        y0, y1 = model(x0, x1)

        # backpropagation
        loss = criterion(y0, y1)
        loss.backward()

        optimizer.step()
        optimizer.zero_grad()

        # calculate the per-dimension standard deviation of the outputs
        # we can use this later to check whether the embeddings are collapsing
        output, _ = y0
        output = output.detach()
        output = torch.nn.functional.normalize(output, dim=1)

        output_std = torch.std(output, 0)
        output_std = output_std.mean()

        # use moving averages to track the loss and standard deviation
        w = 0.9
        avg_loss = w * avg_loss + (1 - w) * loss.item()
        avg_output_std = w * avg_output_std + (1 - w) * output_std.item()

    # the level of collapse is large if the standard deviation of the l2
    # normalized output is much smaller than 1 / sqrt(dim)
    collapse_level = max(0., 1 - math.sqrt(out_dim) * avg_output_std)
    # print intermediate results
    print(f'[Epoch {e:3d}] '
        f'Loss = {avg_loss:.2f} | '
        f'Collapse Level: {collapse_level:.2f} / 1.00')

## Get embeddings

In [ ]:
embeddings = []
filenames = []

# disable gradients for faster calculations
model.eval()
with torch.no_grad():
    for i, (x, _, fnames) in enumerate(dataloader_test):
        # move the images to the gpu
        x = x.to(device)
        # embed the images with the pre-trained backbone
        y = model.backbone(x)
        y = y.squeeze()
        # store the embeddings and filenames in lists
        embeddings.append(y)
        filenames = filenames + list(fnames)

# concatenate the embeddings and convert to numpy
embeddings = torch.cat(embeddings, dim=0)
embeddings = embeddings.cpu().numpy()

In [ ]:
# for plotting
import os
from PIL import Image

import matplotlib.pyplot as plt
import matplotlib.offsetbox as osb
from matplotlib import rcParams as rcp

# for resizing images to thumbnails
import torchvision.transforms.functional as functional

# for clustering and 2d representations
from sklearn import random_projection

In [ ]:
# for the scatter plot we want to transform the images to a two-dimensional
# vector space using a random Gaussian projection
projection = random_projection.GaussianRandomProjection(n_components=2)
embeddings_2d = projection.fit_transform(embeddings)

# normalize the embeddings to fit in the [0, 1] square
M = np.max(embeddings_2d, axis=0)
m = np.min(embeddings_2d, axis=0)
embeddings_2d = (embeddings_2d - m) / (M - m)

In [ ]:
def get_scatter_plot_with_thumbnails():
    """Creates a scatter plot with image overlays.
    """
    # initialize empty figure and add subplot
    fig = plt.figure()
    fig.suptitle('Scatter Plot of the Sentinel-2 Dataset')
    ax = fig.add_subplot(1, 1, 1)
    # shuffle images and find out which images to show
    shown_images_idx = []
    shown_images = np.array([[1., 1.]])
    iterator = [i for i in range(embeddings_2d.shape[0])]
    np.random.shuffle(iterator)
    for i in iterator:
        # only show image if it is sufficiently far away from the others
        dist = np.sum((embeddings_2d[i] - shown_images) ** 2, 1)
        if np.min(dist) < 2e-3:
            continue
        shown_images = np.r_[shown_images, [embeddings_2d[i]]]
        shown_images_idx.append(i)

    # plot image overlays
    for idx in shown_images_idx:
        thumbnail_size = int(rcp['figure.figsize'][0] * 2.)
        path = os.path.join(path_to_data, filenames[idx])
        img = Image.open(path)
        img = functional.resize(img, thumbnail_size)
        img = np.array(img)
        img_box = osb.AnnotationBbox(
            osb.OffsetImage(img, cmap=plt.cm.gray_r),
            embeddings_2d[idx],
            pad=0.2,
        )
        ax.add_artist(img_box)

    # set aspect ratio
    ratio = 1. / ax.get_data_ratio()
    ax.set_aspect(ratio, adjustable='box')


# get a scatter plot with thumbnail overlays
get_scatter_plot_with_thumbnails()

In [ ]:
example_images = [
    '../datasets/logo2k/train/100 Grand Bar/', # water 1
    'S2B_MSIL1C_20200526T101559_N0209_R065_T32SLJ/tile_00527.png', # water 2
    'S2B_MSIL1C_20200526T101559_N0209_R065_T32TNL/tile_00556.png', # land
    'S2B_MSIL1C_20200526T101559_N0209_R065_T31SGD/tile_01731.png', # clouds 1
    'S2B_MSIL1C_20200526T101559_N0209_R065_T32SMG/tile_00238.png', # clouds 2
]


def get_image_as_np_array(filename: str):
    """Loads the image with filename and returns it as a numpy array.

    """
    img = Image.open(filename)
    return np.asarray(img)


def get_image_as_np_array_with_frame(filename: str, w: int = 5):
    """Returns an image as a numpy array with a black frame of width w.

    """
    img = get_image_as_np_array(filename)
    ny, nx, _ = img.shape
    # create an empty image with padding for the frame
    framed_img = np.zeros((w + ny + w, w + nx + w, 3))
    framed_img = framed_img.astype(np.uint8)
    # put the original image in the middle of the new one
    framed_img[w:-w, w:-w] = img
    return framed_img


def plot_nearest_neighbors_3x3(example_image: str, i: int):
    """Plots the example image and its eight nearest neighbors.

    """
    n_subplots = 9
    # initialize empty figure
    fig = plt.figure()
    fig.suptitle(f"Nearest Neighbor Plot {i + 1}")
    #
    example_idx = filenames.index(example_image)
    # get distances to the cluster center
    distances = embeddings - embeddings[example_idx]
    distances = np.power(distances, 2).sum(-1).squeeze()
    # sort indices by distance to the center
    nearest_neighbors = np.argsort(distances)[:n_subplots]
    # show images
    for plot_offset, plot_idx in enumerate(nearest_neighbors):
        ax = fig.add_subplot(3, 3, plot_offset + 1)
        # get the corresponding filename
        fname = os.path.join(path_to_data, filenames[plot_idx])
        if plot_offset == 0:
            ax.set_title(f"Example Image")
            plt.imshow(get_image_as_np_array_with_frame(fname))
        else:
            plt.imshow(get_image_as_np_array(fname))
        # let's disable the axis
        plt.axis("off")


# show example images for each cluster
for i, example_image in enumerate(example_images):
    plot_nearest_neighbors_3x3(example_image, i)